In [1]:
import requests
import json
import datetime
import os

def kakao_login(code):
    url = "https://kauth.kakao.com/oauth/token"
    
    data = {
        "grant_type": "authorization_code",
        "client_id": "11d8ebef889239a83c2db142c8d841a6",
        "redirect_uri": "http://localhost:7777/kakaoLogin",
        "code": code
    }

    response = requests.post(url, data=data)

    if response.status_code != 200:
        print("에러!!!")
    else:
        tokens = response.json()
        print(tokens)
    
    return tokens
        
def save_tokens(filename, tokens):
    with open(filename, "w") as fp:
        json.dump(tokens, fp)
        
# 읽기
def load_tokens(filename):
    with open(filename) as fp:
        tokens = json.load(fp)
        
    return tokens

In [2]:
def getProfile(tokens):
    url = "https://kapi.kakao.com/v2/user/me"

    headers = {
        "Authorization": "Bearer " + tokens['access_token'],
    }

    response = requests.post(url, headers=headers)
    print(response.status_code)
    profile = response.json()

    if response.status_code != 200:
        print("에러!")
    else:
        print("카톡 메시지가 간드아!!!")
    
    return profile

In [ ]:
from flask import Flask, url_for, request, jsonify
from flask_cors import CORS, cross_origin

import json

app = Flask(__name__)
CORS(app)

@app.route('/kakao-login', methods = ['POST'])
def login():
    #Spring에서 code 값 받아오기
    params = json.loads(request.get_data())
    #코드값 code에 저장
    code = params['code']
    KAKAO_LOGIN_TOKEN_SAVED_FILE = "kakao_login/kakao_login_token.json"
    #토큰 요청 후 받아오기
    authorize_tokens = kakao_login(code)
    #토큰 저장
    save_tokens(KAKAO_LOGIN_TOKEN_SAVED_FILE, authorize_tokens)
    #토큰 불러오기
    tokens = load_tokens(KAKAO_LOGIN_TOKEN_SAVED_FILE)
   
    
    #프로필 요청 및 프로필 정보 spring으로 전달
    return getProfile(tokens)


if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [27/May/2022 20:39:55] "POST /kakao-login HTTP/1.1" 200 -


{'access_token': '7NH8WsWVEqgIc2Va6DaUhVlg2fTreZuIxapOZbumCj1zTgAAAYEFUOGo', 'token_type': 'bearer', 'refresh_token': 'hUTlNSn8ecYNpb1tXtFKUaAPEEPM9iRCT1dAkw1ECj1zTgAAAYEFUOGm', 'expires_in': 21599, 'scope': 'account_email profile_nickname', 'refresh_token_expires_in': 5183999}
200
카톡 메시지가 간드아!!!
